In [0]:
import keras
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.engine import Layer
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard 
from keras.models import Sequential, Model
from keras.layers.core import RepeatVector, Permute
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf

Using TensorFlow backend.


In [0]:
import os
import shutil

In [0]:
#reading Train Path
os.chdir('/content/drive/My Drive/Data/new')
path = os.getcwd()
print(path)

/content/drive/My Drive/Data/new


In [0]:
os.rmdir('/content/drive/My Drive/Data/Train_2/second')

In [0]:
os.getcwd()

'/content'

In [0]:
#number of training images
count = 0
for filename in os.listdir('/content/drive/My Drive/Data/Train_2/Train') :
  count+=1
  print (filename)
print(count) 


rnlTwk.jpg
Rot4DK.jpg
rOZe8b.jpg
RO7aMC.jpg
RNZbKm.jpg
ROI2mk.jpg
ROBrDv.jpg
RoCu3I.jpg
RpB7Il.jpg
Rppsxb.jpg
rqv64c.jpg
RQgei7.jpg
RPZ4lY.jpg
RrJtX4.jpg
rPRGIz.jpg
rQopnI.jpg
rrCBMu.jpg
rQC1sd.jpg
rRNUnC.jpg
rSiH2m.jpg
rsIW4Y.jpg
RsXmpp.jpg
RUKnZh.jpg
RtiZ6v.jpg
RtVRTI.jpg
RTP9Ul.jpg
rtL1yT.jpg
Ru58xF.jpg
rulWNr.jpg
RvA6WE.jpg
rvSZgm.jpg
rvbFUV.jpg
RwbG0C.jpg
rW6xmb.jpg
rVmYHn.jpg
rvIHcm.jpg
Rvt9ez.jpg
RwcCaq.jpg
rwdfTK.jpg
RwgEpw.jpg
rWmYTz.jpg
rwPQLv.jpg
rwqSJQ.jpg
rwRWGQ.jpg
rWvdi8.jpg
rWZPDF.jpg
RWurCx.jpg
RWtgdO.jpg
RXPKzh.jpg
rXrU5j.jpg
RYhCDD.jpg
Rx6Ldt.jpg
ryJef1.jpg
Rz7s8e.jpg
ryUj9B.jpg
rXoJJA.jpg
RytSGS.jpg
RYq6CP.jpg
S1EznQ.jpg
S1S62J.jpg
rzaco6.jpg
s24SbS.jpg
s1Plmi.jpg
S1fqCx.jpg
S1wIKi.jpg
s0W7Ri.jpg
S1MwsZ.jpg
S1CkIh.jpg
s2DRcr.jpg
s2M3r8.jpg
s2ifNg.jpg
S2mkrg.jpg
S2ymID.jpg
s3qG0L.jpg
S4aK8k.jpg
S41eRa.jpg
S4bJwF.jpg
S3VZ4a.jpg
S67CEp.jpg
s4jst4.jpg
s644qd.jpg
S5ADlG.jpg
s5hSVS.jpg
s4qJIh.jpg
S5tf5x.jpg
S4OUSi.jpg
s6iKQp.jpg
S6cv4i.jpg
s8eqnd.jpg
s6ork5.jpg
SaHMh5.jpg

In [0]:
import shutil
count = 0
for filename in os.listdir('/content/drive/My Drive/Data/Train_2/Train') :
  if(count<4000):
    print(filename)
    shutil.move('/content/drive/My Drive/Data/Train_2/Train/'+filename, '/content/drive/My Drive/Data/new')
    count+=1
  else:
   break
  

In [0]:
os.mkdir('/content/drive/My Drive/Data/new')

In [0]:
count = len(os.listdir('/content/drive/My Drive/Data/new'))
print(count)
  

4010


In [0]:

# Get images
X = []
for filename in os.listdir('/content/drive/My Drive/Data/new'):
    X.append(img_to_array(load_img('/content/drive/My Drive/Data/new/'+filename)))
X = np.array(X, dtype=float)
Xtrain = 1.0/255*X
print("Execution over")

Execution over


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#Load weights
inception = InceptionResNetV2(weights='imagenet', include_top=True)
inception.graph = tf.get_default_graph()









225214464/225209952 [==============================] - 19s 0us/step


In [0]:
'''X = rgb2lab(1.0/255*image)[:,:,0]
Y = rgb2lab(1.0/255*image)[:,:,1:]
Y /= 128
X = X.reshape(1, 400, 400, 1)
Y = Y.reshape(1, 400, 400, 2)'''

In [0]:
embed_input = Input(shape=(1000,))#Encoder
encoder_input = Input(shape=(256, 256, 1,))
encoder_output = Conv2D(64, (3,3), activation='relu', padding='same', strides=2)(encoder_input)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)#Fusion
fusion_output = RepeatVector(32 * 32)(embed_input) 
fusion_output = Reshape(([32, 32, 1000]))(fusion_output)
fusion_output = concatenate([encoder_output, fusion_output], axis=3) 
fusion_output = Conv2D(256, (1, 1), activation='relu', padding='same')(fusion_output)#Decoder
decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(fusion_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(2, (3, 3), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
model = Model(inputs=[encoder_input, embed_input], outputs=decoder_output)

In [0]:
# Building the neural network
'''model = Sequential()
model.add(InputLayer(input_shape=(None, None, 1)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))'''

In [0]:
def create_inception_embedding(grayscaled_rgb):
    grayscaled_rgb_resized = []
    for i in grayscaled_rgb:
        i = resize(i, (299, 299, 3), mode='constant')
        grayscaled_rgb_resized.append(i)
    grayscaled_rgb_resized = np.array(grayscaled_rgb_resized)
    grayscaled_rgb_resized = preprocess_input(grayscaled_rgb_resized)
    with inception.graph.as_default():
        embed = inception.predict(grayscaled_rgb_resized)
    return embed

In [0]:
# Image transformer
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)#Generate training data
batch_size = 10

In [0]:
def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        grayscaled_rgb = gray2rgb(rgb2gray(batch))
        embed = create_inception_embedding(grayscaled_rgb)
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        X_batch = X_batch.reshape(X_batch.shape+(1,))
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield ([X_batch, create_inception_embedding(grayscaled_rgb)], Y_batch)

In [0]:
#Train model      
model.compile(optimizer='rmsprop', loss='mse')
model.fit_generator(image_a_b_gen(batch_size), epochs=1000, steps_per_epoch=1)


Epoch 1/1000
1/1 [==============================] - 29s 29s/step - loss: 0.0215
Epoch 2/1000
1/1 [==============================] - 12s 12s/step - loss: 0.9647
Epoch 3/1000
1/1 [==============================] - 13s 13s/step - loss: 0.9944
Epoch 4/1000
1/1 [==============================] - 13s 13s/step - loss: 0.9996
Epoch 5/1000
1/1 [==============================] - 13s 13s/step - loss: 1.0725
Epoch 6/1000
1/1 [==============================] - 13s 13s/step - loss: 0.9831
Epoch 7/1000
1/1 [==============================] - 13s 13s/step - loss: 1.0366
Epoch 8/1000
1/1 [==============================] - 13s 13s/step - loss: 1.0092
Epoch 9/1000
1/1 [==============================] - 13s 13s/step - loss: 0.9914
Epoch 10/1000
1/1 [==============================] - 13s 13s/step - loss: 1.0050
Epoch 11/1000
1/1 [==============================] - 13s 13s/step - loss: 1.0162
Epoch 12/1000
1/1 [==============================] - 13s 13s/step - loss: 0.9838
Epoch 13/1000
1/1 [=================

In [0]:
# Finish model
'''model.compile(optimizer='rmsprop',loss='mse')'''

In [0]:
color_me = []
c = 0
for filename in os.listdir('/content/drive/My Drive/Data/new'):
  if(c<10):
    color_me.append(img_to_array(load_img('/content/drive/My Drive/Data/new/'+filename)))
  else:
    break
  c+=1
color_me = np.array(color_me, dtype=float)
gray_me = gray2rgb(rgb2gray(1.0/255*color_me))
color_me_embed = create_inception_embedding(gray_me)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))

In [0]:
os.mkdir('/content/drive/My Drive/Data/result2')

In [0]:
# Test model
output = model.predict([color_me, color_me_embed])
output = output * 128# Output colorizations
for i in range(len(output)):
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = color_me[i][:,:,0]
    cur[:,:,1:] = output[i]
    imsave("/content/drive/My Drive/Data/result2/img_"+str(i)+".png", lab2rgb(cur))

/usr/local/lib/python3.6/dist-packages/skimage/color/colorconv.py:988: UserWarning: Color data out of range: Z < 0 in 54841 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
/usr/local/lib/python3.6/dist-packages/skimage/color/colorconv.py:988: UserWarning: Color data out of range: Z < 0 in 53770 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
/usr/local/lib/python3.6/dist-packages/skimage/io/_io.py:141: UserWarning: /content/drive/My Drive/Data/result2/img_1.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/dist-packages/skimage/color/colorconv.py:988: UserWarning: Color data out of range: Z < 0 in 25552 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
/usr/local/lib/python3.6/dist-packages/skimage/io/_io.py:141: UserWarning: /content/drive/My Drive/Data/result2/img_2.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/loca

In [0]:
'''model.fit(x=X, 
    y=Y,
    batch_size=1,
    epochs=1000)'''

Epoch 1/1000
1/1 [==============================] - 0s 384ms/step - loss: 0.0032
Epoch 2/1000
1/1 [==============================] - 0s 370ms/step - loss: 0.0044
Epoch 3/1000
1/1 [==============================] - 0s 375ms/step - loss: 0.0042
Epoch 4/1000
1/1 [==============================] - 0s 370ms/step - loss: 0.0034
Epoch 5/1000
1/1 [==============================] - 0s 344ms/step - loss: 0.0096
Epoch 6/1000
1/1 [==============================] - 0s 316ms/step - loss: 0.0049
Epoch 7/1000
1/1 [==============================] - 0s 310ms/step - loss: 0.0044
Epoch 8/1000
1/1 [==============================] - 0s 313ms/step - loss: 0.0048
Epoch 9/1000
1/1 [==============================] - 0s 316ms/step - loss: 0.0046
Epoch 10/1000
1/1 [==============================] - 0s 317ms/step - loss: 0.0045
Epoch 11/1000
1/1 [==============================] - 0s 309ms/step - loss: 0.0046
Epoch 12/1000
1/1 [==============================] - 0s 311ms/step - loss: 0.0046
Epoch 13/1000
1/1 [======

In [0]:
print(model.evaluate(X, Y, batch_size=1))
output = model.predict(X)
output *= 128
# Output colorizations
cur = np.zeros((400, 400, 3))
cur[:,:,0] = X[0][:,:,0]
cur[:,:,1:] = output[0]
imsave("img_result.png", lab2rgb(cur))
imsave("img_gray_version.png", rgb2gray(lab2rgb(cur)))

1/1 [==============================] - 0s 88ms/step


0.00299252406693995


https://medium.com/@rgrgrajat1/colorize-your-black-white-world-with-deep-learning-studio-cee66d35b9ff